In [4]:
%pip install gradio

  Using cached gradio-5.29.1-py3-none-any.whl.metadata (16 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached gradio_client-1.10.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached huggingface_hub-0.31.2-py3-none-any.whl.metadata (13 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached typer-0.15.4-py3-none-any.whl.metadata (15 kB)
Using cached gradio-5.29.1-py3-none-any.whl (54.1 MB)
Using cached gradio_client-1.10.1-py3-none-any.whl (323 kB)
Using cached fastapi-0.115.12-py3-none-any.whl (95 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached huggingface_hub-0.31.2-py3-none-any.whl (484 kB)
Using cached safehttpx-0.1.6-py3-none-any.whl (8.7 kB)
Using cached typer-0.15.4-py3-none-any.whl (45 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 32] Der Prozess kann nicht auf die Datei zugreifen, da sie von einem anderen Prozess verwendet wird: 'c:\\Users\\leonie\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\huggingface_hub\\inference\\_providers\\cerebras.py'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import gradio as gr

# ---------- Page 1: 3 Buttons ----------
def button_clicked(choice):
    return f"You clicked: {choice}"

with gr.Blocks() as page1:
    gr.Markdown("## Page 1: Choose a Button")
    with gr.Row():
        btn1 = gr.Button("Button 1")
        btn2 = gr.Button("Button 2")
        btn3 = gr.Button("Button 3")
    output1 = gr.Textbox(label="Result")
    
    btn1.click(fn=lambda: button_clicked("Button 1"), outputs=output1)
    btn2.click(fn=lambda: button_clicked("Button 2"), outputs=output1)
    btn3.click(fn=lambda: button_clicked("Button 3"), outputs=output1)

# ---------- Page 2: Simple Chat ----------
chat_history = []

def chat_response(message, history):
    history = history or []
    response = f"You said: {message}"
    history.append((message, response))
    return "", history

with gr.Blocks() as page2:
    gr.Markdown("## Page 2: Chat with the App")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your Message")
    send_btn = gr.Button("Send")
    
    send_btn.click(fn=chat_response, inputs=[msg, chatbot], outputs=[msg, chatbot])

# ---------- Page 3: Dummy Text ----------
with gr.Blocks() as page3:
    gr.Markdown("## Page 3: Dummy Text")
    gr.Textbox(value="This is some dummy text. Hello world!", lines=5, label="Note")

# ---------- Main App with Tabs ----------
with gr.Blocks() as demo:
    with gr.Tab("Buttons Page"):
        page1.render()
    with gr.Tab("Chat Page"):
        page2.render()
    with gr.Tab("Text Page"):
        page3.render()

# Launch the app
demo.launch()


c:\Users\leonie\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\leonie\AppData\Local\Temp\ipykernel_17060\450893732.py:30: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [3]:
import gradio as gr

# Topics
topics = [
    "Informations",
    "Risks and Complications",
    "Concerns & Questions",
    "Day Preparation",
    "Agenda of the Day",
    "Aftercare & Notifications",
    "Documents & Medical History",
    "Download PDF for Patient"
]

# Dummy Chatbot Function
def respond(message, history, topic):
    history = history or []
    response = f"[{topic}] Echo: {message}"
    history.append((message, response))
    return history, ""

with gr.Blocks() as demo:
    selected_topic = gr.State("")
    current_page = gr.State("page1")

    with gr.Row():
        gr.Markdown("## Patient Assistant")
        settings_button = gr.Button("⚙️ Accessibility Settings")

    settings_button.click(lambda: print("Settings clicked"), outputs=[])

    with gr.Column(visible=True) as page1:
        gr.Markdown("### Please choose a topic")

        colors = [
            "#F9C89B", "#A3BCE2", "#F5A9B8", "#B6D3F2",
            "#C3E7C4", "#F0B3B3", "#FBE9A0", "#D4D4DC"
        ]

        output = gr.Textbox(label="Result", lines=2, visible=False)

        for i in range(0, len(topics), 2):
            with gr.Row():
                for j in range(2):
                    if i + j < len(topics):
                        gr.Button(topics[i + j],
                                  elem_id=f"button-{i+j}",
                                  elem_classes="colored-button").click(
                            lambda t=topics[i + j]: (t, "page2"), None, [selected_topic, current_page]
                        )

    with gr.Column(visible=False) as page2:
        topic_heading = gr.Markdown("")
        chatbot = gr.Chatbot()
        msg = gr.Textbox(label="Enter your message")
        msg.submit(respond, [msg, chatbot, selected_topic], [chatbot, msg])
        back_button = gr.Button("⬅️ Back")
        back_button.click(lambda: "page1", None, current_page)

    # Toggle Page Display
    def render_page(page, topic):
        return {
            page1: gr.update(visible=page == "page1"),
            page2: gr.update(visible=page == "page2"),
            topic_heading: gr.update(value=f"## Chat about: {topic}" if topic else "")
        }

    current_page.change(render_page, [current_page, selected_topic], [page1, page2, topic_heading])

    # CSS Styles to match your image
    gr.HTML("""
    <style>
    .colored-button {
        width: 220px;
        height: 80px;
        font-size: 16px;
        font-weight: bold;
        color: black;
        border-radius: 16px;
        margin: 6px;
        border: none;
        cursor: pointer;
    }
    #button-0 { background-color: #F9C89B; }
    #button-1 { background-color: #A3BCE2; }
    #button-2 { background-color: #F5A9B8; }
    #button-3 { background-color: #B6D3F2; }
    #button-4 { background-color: #C3E7C4; }
    #button-5 { background-color: #F0B3B3; }
    #button-6 { background-color: #FBE9A0; }
    #button-7 { background-color: #D4D4DC; }
    </style>
    """)

demo.launch()


C:\Users\leonie\AppData\Local\Temp\ipykernel_17060\1493185245.py:54: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
